# Enable distributed tracing with OpenTelemetry.

When using distributed agents, it is useful to gather information from all the involved components. In this cookbook, we wil leverage on using A2A agents as tools as demonstrated in ["Use an Agent as a tool for another agent"](./../a2a_as_tool) to obtain traces from different components and have a consistent view of a distirbuted trace.

First, a Grafana Tempo instance needs to be running in your environment. A good starting point is the [quick start for tempo](https://grafana.com/docs/tempo/latest/getting-started/docker-example/), removing the `k6-tracing` component.

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [ ]:
%pip install 'any-agent[a2a]'

import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
import httpx
from getpass import getpass

# This notebook communicates with OpenAI GPT models using the OpenAI API.
if "OPENAI_API_KEY" not in os.environ:
    print("OPENAI_API_KEY not found in environment!")
    api_key = getpass("Please enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY set for this session!")
else:
    print("OPENAI_API_KEY found in environment.")
    
from opentelemetry.trace import get_tracer_provider, TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, BatchSpanProcessor
from opentelemetry.sdk.trace.export import ConsoleSpanExporter
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

from any_agent import AgentConfig, AnyAgent, AgentFramework
from any_agent.tools import a2a_tool_async
from any_agent.serving import A2AServingConfig


In [ ]:
agent_framework = AgentFramework.OPENAI
agent_model = "gpt-4.1-nano"

# Add another trace exporter
tp = get_tracer_provider()
full_exporter = ConsoleSpanExporter()
http_exporter = OTLPSpanExporter(endpoint="http://localhost:4318/v1/traces")
tp.add_span_processor(SimpleSpanProcessor(http_exporter))

main_agent = None
served_agent = None
served_task = None
served_server = None

async def wait_for_server_async(
    server_url: str, max_attempts: int = 20, poll_interval: float = 0.5
):
    attempts = 0

    async with httpx.AsyncClient() as client:
        while True:
            try:
                # Try to make a basic GET request to check if server is responding
                await client.get(server_url, timeout=1.0)
                return  # noqa: TRY300
            except (httpx.RequestError, httpx.TimeoutException):
                # Server not ready yet, continue polling
                pass

            await asyncio.sleep(poll_interval)
            attempts += 1
            if attempts >= max_attempts:
                msg = f"Could not connect to {server_url}. Tried {max_attempts} times with {poll_interval} second interval."
                raise ConnectionError(msg)

try:
    tool_agent_endpoint = "tool_agent"
    test_port = 9999

    # DATE AGENT

    import datetime

    def get_datetime() -> str:
        """Return the current date and time"""
        return str(datetime.datetime.now())

    date_agent_description = "Agent that can return the current date."
    date_agent_cfg = AgentConfig(
        instructions="Use the available tools to obtain additional information to answer the query.",
        name="date_agent",
        model_id=agent_model,
        description=date_agent_description,
        tools=[get_datetime],
    )
    date_agent = await AnyAgent.create_async(
        agent_framework=agent_framework,
        agent_config=date_agent_cfg,
    )

    served_agent = date_agent
    (served_task, served_server) = await served_agent.serve_async(
        serving_config=A2AServingConfig(
            port=test_port,
            endpoint=f"/{tool_agent_endpoint}",
            log_level="info",
        )
    )
    server_url = f"http://localhost:{test_port}/{tool_agent_endpoint}"
    await wait_for_server_async(server_url)

    # Search agent is ready for card resolution

    main_agent_cfg = AgentConfig(
        instructions="Use the available tools to obtain additional information to answer the query.",
        name="main_agent",
        model_id=agent_model,
        description="The orchestrator that can use other agents via tools using the A2A protocol.",
        tools=[await a2a_tool_async(server_url, http_kwargs={"timeout": 10.0})],
    )

    main_agent = await AnyAgent.create_async(
        agent_framework=agent_framework,
        agent_config=main_agent_cfg,
    )

    DATE_PROMPT = (
        "What date and time is it right now? "
        "In your answer please include the year, month, day, and time. "
        "Example answer could be something like 'Today is December 15, 2024'"
    )
    agent_trace = await main_agent.run_async(DATE_PROMPT)

    print(agent_trace.final_output)

finally:
    if served_server:
        served_server.should_exit = True
        await served_task
